# Product Matching Book Analysis Using K-means
### Kelompok 13
1. Yolanda R.M Manurung (12S17017)
2. Stella Sitinjak      (12S17027)
3. Ekis Naomi Lasma     (12S17059)


### Dataset
Data yang digunakan pada proyek ini berasal dari kaggle (https://www.kaggle.com/gapple/goodreads-book-reviews-20k)

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
%matplotlib inline
# import sys
# reload(sys)
# sys.setdefaultencoding('utf-8')
import re
import csv
import ast
import operator
import pandas as pd
import seaborn as sns
import string
from string import digits
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import sent_tokenize, LineTokenizer, RegexpTokenizer

from collections import Counter
from matplotlib import cm
from itertools import cycle, islice
import matplotlib as mpl
import matplotlib.pyplot as plt
from subprocess import check_output
from wordcloud import WordCloud


from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

from pandas import *
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn import linear_model
import numpy as np

### Cross-Industry Standard Process for Data Mining (CRISP-DM)

### 1. Data understanding

In [3]:
review = pd.read_csv('all_data.csv')
print ('Jumlah data =', len(review))
review.head()

Jumlah data = 21559


,book_title,Book_series,book_series_url,book_image,book_image_url,book_rating,book_author,author_url,genre,reviewer_name,reviewer_url,reviewer_image,reviewer_image_url,review,ID
0,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",emma,https://www.goodreads.com/user/show/32879029-emma,https://images.gr-assets.com/users/1520568130p...,https://www.goodreads.com/user/show/32879029-emma,202.\r\nThat’s the number of times that the wo...,1
1,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Em (RunawayWithDreamthieves),https://www.goodreads.com/user/show/42130592-e...,https://images.gr-assets.com/users/1538507799p...,https://www.goodreads.com/user/show/42130592-e...,I need a black silk robe and a balcony to wear...,2
2,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Cait • A Page with a View,https://www.goodreads.com/user/show/47412532-c...,https://images.gr-assets.com/users/1523134490p...,https://www.goodreads.com/user/show/47412532-c...,THIS BOOK TOTALLY WRECKED ME. In the best poss...,3
3,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Natalie Monroe,https://www.goodreads.com/user/show/6688207-na...,https://images.gr-assets.com/users/1339730817p...,https://www.goodreads.com/user/show/6688207-na...,"3.75 stars\r\n""Rhys stepped off the stairs and...",4
4,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Emily May,https://www.goodreads.com/user/show/4622890-em...,https://images.gr-assets.com/users/1528390751p...,https://www.goodreads.com/user/show/4622890-em...,I wish so much that I could tell you I loved t...,5


In [4]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21559 entries, 0 to 21558
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   book_title          21559 non-null  object 
 1   Book_series         8780 non-null   object 
 2   book_series_url     8780 non-null   object 
 3   book_image          21557 non-null  object 
 4   book_image_url      21557 non-null  object 
 5   book_rating         21559 non-null  float64
 6   book_author         21559 non-null  object 
 7   author_url          21559 non-null  object 
 8   genre               21438 non-null  object 
 9   reviewer_name       21534 non-null  object 
 10  reviewer_url        21534 non-null  object 
 11  reviewer_image      21534 non-null  object 
 12  reviewer_image_url  21534 non-null  object 
 13  review              20817 non-null  object 
 14  ID                  21559 non-null  int64  
dtypes: float64(1), int64(1), object(13)
memory usage: 2.5

In [5]:
review.columns

Index(['book_title', 'Book_series', 'book_series_url', 'book_image',
       'book_image_url', 'book_rating', 'book_author', 'author_url', 'genre',
       'reviewer_name', 'reviewer_url', 'reviewer_image', 'reviewer_image_url',
       'review', 'ID'],
      dtype='object')

In [6]:
review.isna().sum()

book_title                0
Book_series           12779
book_series_url       12779
book_image                2
book_image_url            2
book_rating               0
book_author               0
author_url                0
genre                   121
reviewer_name            25
reviewer_url             25
reviewer_image           25
reviewer_image_url       25
review                  742
ID                        0
dtype: int64

### 2. Business understanding

### 3. Data Cleaning and Preperation

In [7]:
review.shape

(21559, 15)

Drop Null Value

In [17]:
review = review.dropna()

In [18]:
review.shape

(8600, 15)

In [19]:
data_train, data_test = train_test_split(review, test_size=0.2)
data_train_copy = data_train
print("%d items in training data, %d in test data" % (len(data_train), len(data_test)))

6880 items in training data, 1720 in test data


In [20]:
df  = pd.DataFrame(review[['book_title','book_rating','genre','review']])

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8600 entries, 0 to 21558
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   book_title   8600 non-null   object 
 1   book_rating  8600 non-null   float64
 2   genre        8600 non-null   object 
 3   review       8600 non-null   object 
dtypes: float64(1), object(3)
memory usage: 335.9+ KB


In [22]:
df.describe()

,book_rating
count,8600.000000
mean,4.141544
std,0.248665
min,3.360000
25%,3.980000
50%,4.150000
75%,4.310000
max,4.690000


In [23]:
df['review'].head(20)

0     202.\r\nThat’s the number of times that the wo...
1     I need a black silk robe and a balcony to wear...
2     THIS BOOK TOTALLY WRECKED ME. In the best poss...
3     3.75 stars\r\n"Rhys stepped off the stairs and...
4     I wish so much that I could tell you I loved t...
5     I’m wrecked. Because a Court of Wings and Ruin...
6     Yay! Another one of my books won 2017 GR award...
7     Congratulations-winner of Best Young Adult Fan...
8     In these early days after the release, I'll sa...
9     Glorious, thrilling and completely enthralling...
10    *Do I need to remind you to be civil? No, I do...
11    "And will you come with me? On this adventure ...
12    When the book you're highly anticipating has n...
13    BIBLIOBABBLE- https://youtu.be/y5UVdj8avvs (re...
14    I. CAN'T. HANDLE. THIS. WHAT???\r\nRTC\r\n----...
15    Reread x 2.\r\nMy heart is in pain and full of...
16    You know you're a dedicated fan when you mark ...
17    4.25/5 stars\r\n"Remember that you are a w

Remove Punctuation

In [24]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [25]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

df['book_title'] = df['book_title'].apply(lambda x: remove_punct(x))
df['review'] = df['review'].apply(lambda x: remove_punct(x))
df.head(10)

,book_title,book_rating,genre,review
0,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",\r\nThat’s the number of times that the word “...
1,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",I need a black silk robe and a balcony to wear...
2,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",THIS BOOK TOTALLY WRECKED ME In the best possi...
3,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",stars\r\nRhys stepped off the stairs and took...
4,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",I wish so much that I could tell you I loved t...
5,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",I’m wrecked Because a Court of Wings and Ruin ...
6,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",Yay Another one of my books won GR award ❤️\r...
7,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",Congratulationswinner of Best Young Adult Fant...
8,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",In these early days after the release Ill say ...
9,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",Glorious thrilling and completely enthralling ...


Case Folding

In [26]:
df['review'] = df['review'].str.lower()
df['book_title'] = df['book_title'].str.lower()

df.head()

,book_title,book_rating,genre,review
0,a court of wings and ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",\r\nthat’s the number of times that the word “...
1,a court of wings and ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",i need a black silk robe and a balcony to wear...
2,a court of wings and ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",this book totally wrecked me in the best possi...
3,a court of wings and ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",stars\r\nrhys stepped off the stairs and took...
4,a court of wings and ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",i wish so much that i could tell you i loved t...


Stopwords Removing

In [27]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [28]:
cachedStopWords = set(stopwords.words("english"))
for i in range (len(df)):
    meaningful = df['review'].iloc[i]
    meaningful = " ".join([word for word in meaningful.split() if word not in cachedStopWords])
    df['review'].iloc[i]=meaningful
print ('Data trainstop words removing... ')
df.head(20)

Data trainstop words removing... 


,book_title,book_rating,genre,review
0,a court of wings and ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",that’s number times word “mate” hellish forms ...
1,a court of wings and ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",need black silk robe balcony wear i’m gonna dr...
2,a court of wings and ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",book totally wrecked best possible way i’m thr...
3,a court of wings and ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",stars rhys stepped stairs took hand night triu...
4,a court of wings and ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",wish much could tell loved book didnt thinking...
5,a court of wings and ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",i’m wrecked court wings ruin best conclusion c...
6,a court of wings and ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",yay another one books gr award ❤️ omgthis book...
7,a court of wings and ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",congratulationswinner best young adult fantasy...
8,a court of wings and ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",early days release ill say little spoil nothin...
9,a court of wings and ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",glorious thrilling completely enthralling acow...


In [29]:
df.to_csv("dfReview.csv")

In [30]:
df = pd.read_csv("dfReview.csv")

4. Training Models